In [1]:
# Because our images are fairly small, it will be faster to just cache them all in RAM rather than perform IO reads at runtime
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

LLabels = [] # All the labels, in order
LImages = [] # All the images, in order

for root, dirs, files in os.walk("plots"):
   for name in files:
      shortnameSplitted = name.rstrip(".jpg").split("_")
      LLabels.append((float(shortnameSplitted[1]),float(shortnameSplitted[2])))
      im = Image.open(os.path.join(root, name))
      #print(np.shape(np.array(im)))
      #input()
      #r, g, b = np.array(im).T
      LImages.append(np.array(im))
      plt.show()

print(len(LImages))
print(len(LLabels))

900
900


In [11]:
from sklearn.model_selection import KFold

n_split=3
x_train, y_train, x_test, y_test = [[],[],[],[]], [[],[],[],[]], [[],[],[],[]], [[],[],[],[]]
n = 0
for train_index,test_index in KFold(n_split).split(LImages):
    for k in train_index:
        x_train[n].append(LImages[k])
        y_train[n].append(LLabels[k])
    for k in test_index:
        x_test[n].append(LImages[k])
        y_test[n].append(LLabels[k])
    n+=1
    



600
300


In [25]:
import tensorflow as tf
import tensorflow.python.keras as keras
from keras.applications.resnet50 import ResNet50
from keras.layers import Dense

model = ResNet50(weights = 'imagenet', include_top = False)

x=model.output
x=Dense(2048,activation='relu')(x) 
x=Dense(512,activation='relu')(x) 
preds=Dense(2,activation='softmax')(x) 

model_piche=keras.Model(inputs=model.input,outputs=preds) 

for layer in model.layers:
    layer.trainable=False

model_piche.compile(loss='mean_squared_error', optimizer='adam')
estimator = keras.KerasRegressor(build_fn=model_piche, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, LImages, LLabels, cv=kfold)
print(results.mean(), results.std())


AttributeError: 'tuple' object has no attribute 'layer'